In [28]:
using PCT

## $x^{\dagger} A x$

In [29]:
f, _ = @pct (A::Her) -> pullback((x::CV) -> sum((i, j), x(i)' * A(i, j) * x(j))); f

(A) -> 
    𝒫((x) -> 
        ∑((i, j), (x(j)⋅A(i, j)⋅x(i)')))

In [30]:
df = vdiff(f; settings=symmetry_settings)

(A) -> 
    (x) -> 
        (_d) -> 
            ∑((j), (x(j)⋅A(_d, j)⋅2.0))

## Trace of Product

In [31]:
f, _ = @pct begin
    tr = (X::CM) -> sum(i, X(i, i))
    matmul = (A::CM, B::CM) -> (i::Z, j::Z) -> sum(n, A(i, n) * B(n, j))
    pullback((A::CM, B::CM) -> tr(matmul(A, B) + matmul(B, A)))
end; f

let 
    tr = (X) -> 
        ∑((i), X(i, i))
    matmul = (A, B) -> 
        (i, j) -> 
            ∑((n), (A(i, n)⋅B(n, j)))
    𝒫((A, B) -> 
        tr((matmul(A, B)+matmul(B, A))))
end

In [32]:
vdiff(eval_all(f); settings=symmetry_settings)

(A, B) -> 
    (_d, _d_1) -> 
        (B(_d_1, _d)'⋅2.0), (_d, _d_1) -> 
        (A(_d_1, _d)'⋅2.0)

## Conjugate Gradient

In [33]:
f, _ = @pct (A::Sym, r::RV, p::RV, b::RV, x::RV) -> begin
    E = (x::RV) -> sum((i, j), 0.5 * x(i) * A(i, j) * x(j)) + sum(i, x(i) * b(i))
    pullback((α::R, β::R) -> E((i::Z) -> x(i) + α * (r(i) + β * p(i))))
end; f

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((i), (x(i)⋅b(i)))+∑((i, j), (x(i)⋅x(j)⋅A(i, j)⋅0.5)))
        𝒫((α, β) -> 
            E((i) -> 
                (x(i)+(α⋅(r(i)+(β⋅p(i)))))))
    end

In [34]:
df = vdiff(f)

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((i), (x(i)⋅b(i)))+∑((i, j), (x(i)⋅x(j)⋅A(i, j)⋅0.5)))
        (α, β) -> 
            ∑((i), (𝒫(E)((i) -> 
                (x(i)+(α⋅(r(i)+(β⋅p(i))))), 1)(i)⋅(r(i)+(β⋅p(i))))), ∑((i), (α⋅𝒫(E)((i) -> 
                (x(i)+(α⋅(r(i)+(β⋅p(i))))), 1)(i)⋅p(i)))
    end

In [35]:
df = vdiff(eval_all(f); settings=symmetry_settings)

(A, r, p, b, x) -> 
    (α, β) -> 
        (∑((i), (b(i)⋅(r(i)+(β⋅p(i)))))+∑((i, j), (A(i, j)⋅(x(j)+(α⋅(r(j)+(β⋅p(j)))))⋅(r(i)+(β⋅p(i)))))), (∑((i), (α⋅b(i)⋅p(i)))+∑((i, j), (α⋅p(j)⋅A(i, j)⋅(x(i)+(α⋅(r(i)+(β⋅p(i))))))))

## Lagrangian Mechanics

In [36]:
_, ctx = @pct begin
    @space X begin
        type = (RF, ) -> RF
        linear = true
    end
end; 

In [37]:
f = @pct _ ctx (L::RO, D::X) -> pullback((r::RF) -> sum((t::R), L(r(t), D(r)(t)))); f

(L, D) -> 
    𝒫((r) -> 
        ∑((t), L(r(t), D(r)(t))))

In [38]:
vdiff(f; settings=custom_settings(:clench_sum=>true))

(L, D) -> 
    (r) -> 
        (D'((_d) -> 
            𝒫((_z_1) -> 
                L(r(_d), _z_1))(D(r)(_d), 1))+(_d) -> 
            𝒫((_z_2) -> 
                L(_z_2, D(r)(_d)))(r(_d), 1))

## Hartree Fock Gradient and Hessian Vector Product

In [39]:
_, ctx = @pct begin    
    @space T begin
        type = (I, I, I, I) -> C
        symmetries = (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))
    end
end;

In [40]:
f = @pct _ ctx (J::T) -> pullback((C::CM) -> sum((i, j, p, q, r, s),
    C(p, i)' * C(q, i) * C(r, j)' * C(s, j) * J(p, q, r, s)))

(J) -> 
    𝒫((C) -> 
        ∑((i, j, p, q, r, s), (J(p, q, r, s)⋅C(s, j)⋅C(q, i)⋅C(p, i)'⋅C(r, j)')))

In [41]:
df = vdiff(f; settings=symmetry_settings)

(J) -> 
    (C) -> 
        (_d, _d_1) -> 
            ∑((i, p, q, r), (J(_d, r, q, p)⋅C(p, i)⋅C(r, _d_1)⋅C(q, i)'⋅4.0))

In [42]:
df |> fc |> decompose |> pp |> simplify |> first

(C, _k) -> 
    (_a_2, _a_3) -> 
        (∑((_d, i, p, q), (_k(_d, _a_3)⋅C(q, i)⋅J(_d, _a_2, q, p)'⋅C(p, i)'⋅4.0))+∑((_d, _d_1, q, r), (_k(_d, _d_1)⋅C(q, _a_3)⋅J(_d, r, q, _a_2)'⋅C(r, _d_1)'⋅4.0))+∑((_d, _d_1, p, r), (J(_d, r, _a_2, p)⋅C(p, _a_3)⋅C(r, _d_1)⋅_k(_d, _d_1)'⋅4.0)))

## Foster Boys

In [43]:
f, _ = @pct (R::Her, R2::Her) -> pullback((C::CM) -> begin 
        r = (i::Z) -> sum((p, q), R(p, q) * C(i, p)' * C(i, q))
        sum(i, sum((p, q), R2(p, q) * C(i, p)' * C(i, q)) - r(i)' * r(i))
    end); f

(R, R2) -> 
    𝒫((C) -> 
        let 
            r = (i) -> 
                ∑((p, q), (R(p, q)⋅C(i, q)⋅C(i, p)'))
            ∑((i), ((r(i)⋅r(i)'⋅-1.0)+∑((p, q), (R2(p, q)⋅C(i, q)⋅C(i, p)'))))
        end)

In [44]:
df = vdiff(eval_all(f); settings=symmetry_settings)

(R, R2) -> 
    (C) -> 
        (_d, _d_1) -> 
            (∑((p), (R2(_d_1, p)⋅C(_d, p)⋅2.0))+∑((p, _i, q), (R(_d_1, p)⋅R(q, _i)⋅C(_d, p)⋅C(_d, _i)⋅C(_d, q)'⋅-4.0)))

## Localized Wannier Functions

In [45]:
_, ctx = @pct begin
    @domain BZ begin
        base = I
        periodic = true
    end
    
    @domain X begin
        symmetric = true
        contractable = false
    end
    
    @space Mmn begin
        type = (Z, Z, BZ, BZ) -> C
        symmetries = (((2, 1, 4, 3), :conj),)
    end

    @space SV begin
        type = (I,) -> C
        symmetries = (((1,), :ineg),)
    end
    
    @space Gauge begin
        type = (Z, Z, BZ) -> C
    end
end;

In [46]:
f = @pct _ ctx (S::Mmn, w::SV) -> pullback((U::Gauge) -> 
    begin
        ρ = (n::Z, b::X) -> sum((k::BZ, p, q), U(p, n, k)' * S(p, q, k, k + b) * U(q, n, k + b))
        sum((n, b::X), ρ(n, b)' * ρ(n, b))
    end)

(S, w) -> 
    𝒫((U) -> 
        let 
            ρ = (n, b) -> 
                ∑((k, p, q), (U(q, n, (b+k))⋅S(p, q, k, (b+k))⋅U(p, n, k)'))
            ∑((n, b), (ρ(n, b)⋅ρ(n, b)'))
        end)

In [47]:
# df = vdiff(eval_all(f); settings=custom_settings(:logging=>true; preset = symmetry_settings))
df = vdiff(eval_all(f); settings=symmetry_settings)

(S, w) -> 
    (U) -> 
        (_d, _d_1, _d_2) -> 
            ∑((b, p, k, _i, q), (U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅S(_i, q, (b+k), k)⋅U(_i, _d_1, (b+k))'⋅4.0))

In [48]:
f, _ = @pct begin
    x = 1
    y = (m::R) -> m - 2
    (j::I) ->  x+ y(j)
end; f

let 
    x = 1
    y = (m) -> 
        (m+-2.0)
    (j) -> 
        (y(j)+x)
end

In [26]:
f, _ = @pct begin
    y = (m::R) -> m - 2
    (j::I ∈ (1, 2)) -> y(j)
end; f

let 
    y = (m) -> 
        (m+-2.0)
    (j ∈ (1, 2)) -> 
        y(j)
end